In [ ]:
# Import Libraries
import time

from ImportData2D import get_label, getFilename, get_file_name
from stlSlicer import getSlicer, slicecoor, rotatestl
from imgSave import saveplot
import numpy as np

np.random.seed(125)


In [ ]:
label = getLabel("Occ_Sum", "median", True, False,False)
print(type(label[0]))
label = getLabel("Occ_Sum", "median", True, True,False)
print(type(label[0][0]))

In [ ]:
import os
test_name = "prep_test.tfrecords"
test_name = os.path.join("data", test_name)
print(test_name)

In [ ]:
import os
A = './data/preparation_181_data_test_eval_address.txt'
print(os.path.abspath(A))
print(os.path.isfile(A))

In [ ]:
example_grouped_address = ['/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_0_84140','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_95138_0','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_304138_45']
grouped_address = [1,2,3]
# define an empty list
train_address = []

# open file and read the content in a list
with open(A, 'r') as filehandle:  
    for line in filehandle:
        # remove linebreak which is the last character of the string
        current_name = line[:-1]
        # print(current_name)
        for i, name in enumerate(example_grouped_address):
            
            if current_name in name:
                train_address.append(grouped_address[i])
                grouped_address.remove(grouped_address[i])
                example_grouped_address.remove(example_grouped_address[i])
                

In [ ]:
print(train_address)

In [ ]:
import ast
B = ast.literal_eval(error_file_names[0])
print(B[0])

In [ ]:
train_address = ['abc_0_0','abc_0_90','abc_0_45','abc_0_135','abc_1_0','abc_1_45','abc_1_90','abc_1_70']

In [ ]:
train_address.sort()
print([s for s in train_address if 'abc_0' in s])

In [ ]:
import tensorflow as tf
import csv
A = {'learning_rate':0.00009123,'keep_prob':0.25, 'activation':tf.nn.relu, 'channels':[16,16,32]}
# A = {'a':'b','c':'d'}

with open("output"+".csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    for key, val in A.items():
        writer.writerow([key, val])


In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# You can add the parameter data_home to wherever to where you want to download your data
import tensorflow as tf
import numpy as np
((train_data, train_labels),(eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()
train_data = train_data / np.float32(255)
train_data = train_data.reshape(train_data.shape[0], -1)
print(train_data.shape)
train_labels = train_labels.astype(np.int32)  # not required
print(train_labels.shape)
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split(
    train_data, train_labels, test_size=1/7.0, random_state=0)
print(train_img.shape)

In [ ]:
# Standardizing the Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train_img)

# Apply transform to both the training set and the test set.
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
pca = PCA(.95)
pca.fit(train_img)
pca.n_components_
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

# all parameters not specified are set to their defaults
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')

logisticRegr.fit(train_img, train_lbl)

# Returns a NumPy Array
# Predict for One Observation (image)
logisticRegr.predict(test_img[0].reshape(1,-1))

# Predict for Multiple Observations (images) at Once
logisticRegr.predict(test_img[0:10])

In [ ]:
score = logisticRegr.score(test_img, test_lbl)
print(score)

In [ ]:
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np

# Fixed parameter
numdegree = 4  # Number of rotations
image_height = 240  # Used for cropping
image_width = 360  # Used for cropping


# Import tfrecord to dataset
def deserialize(example):
    feature = {'label': tf.FixedLenFeature([], tf.int64)}
    for i in range(numdegree):
        feature['img' + str(i)] = tf.FixedLenFeature([], tf.string)
    return tf.parse_single_example(example, feature)


def decode(data_dict):
    if numdegree != 4:
        raise Exception('Number of degree specified is not compatible, edit code')
    # Create initial image, then stacking it
    image_decoded = []

    # Stacking the rest
    for i in range(0, numdegree):
        img = data_dict['img' + str(i)]
        file_decoded = tf.image.decode_png(img, channels=1)
        file_cropped = tf.squeeze(tf.image.resize_image_with_crop_or_pad(file_decoded, image_height, image_width))
        image_decoded.append(file_cropped)

    image_stacked = tf.stack([image_decoded[0], image_decoded[1], image_decoded[2], image_decoded[3]], axis=2)
    image_stacked = tf.cast(image_stacked, tf.float32)
    label = data_dict['label']
    # output = (image_stacked, label)
    # return {'images': image_stacked, 'label': label}  # Output is [Channel, Height, Width]
    return image_stacked, label


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def get_data_from_path(data_path):
    dataset = tf.data.TFRecordDataset(data_path)
    #print(dataset)
    dataset = dataset.map(deserialize)
    dataset = dataset.map(decode)
    
    max_elems = np.iinfo(np.int64).max
    dataset = dataset.batch(1000, drop_remainder=False)
    whole_dataset_tensors = tf.data.experimental.get_single_element(dataset)
    with tf.Session() as sess:
        whole_dataset_arrays = sess.run(whole_dataset_tensors)
    return whole_dataset_arrays
    
    '''
    iterator = dataset.make_one_shot_iterator()
    
    next_example, next_label = iterator.get_next()
    with tf.Session() as sess:
        example = sess.run(next_example)
        label = sess.run(next_label)
    return example, label
    '''

In [ ]:
data_path = './data/preparation_181_data_Taper_Sum_median_eval.tfrecords'
arrays = get_data_from_path(data_path)
# example, label = get_data_from_path(data_path)
# print(example[0])
print((arrays[1]))

In [ ]:
ex, label = get_data_from_path(data_path)
print(label)

In [ ]:
ex, label = get_data_from_path(data_path)
print(label)

In [ ]:
import csv

csvData = [['Person', 'Age'], ['Peter', '22'], ['Jasmine', '21'], ['Sam', '24']]
print(type(csvData))
print((csvData))
print(type(csvData[0]))
print((csvData[0]))
'''
with open('person.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)

csvFile.close()'''

In [ ]:
csvData = [(-0.53103447, [1e-05, 0.0, '1', 1]), (-0.53103447, [4.315202424267864e-05, 0.0, '1', 1]), (-0.53103447, [6.388961292348475e-05, 0.04720173848853115, '1', 4]), (-0.53103447, [0.0001540877645803547, 0.35419707255064065, '1', 1])]
newData = []
for i in csvData:
    data = {'value': i[0],
           'learning_rate': i[1][0],
           'dropout_rate': i[1][1],
           'activation': i[1][1],
           'channels': i[1][1],}
    newData.append(data)
print(type(csvData))
print((csvData))
#print(type(csvData[0]))
# print((csvData[0]))


with open('person.csv', 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames = ['value','learning_rate','dropout_rate', 'activation', 'channels'])
    writer.writeheader()
    writer.writerows(newData)



In [ ]:
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np
import os
import argparse
from shutil import copy2
import csv
import datetime

from protobuf_helper import protobuf_to_list, protobuf_to_channels
from proto import tooth_pb2
from google.protobuf import text_format

import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args

from cifar10_model import my_model
from cifar10_get_data import train_input_fn, eval_input_fn, get_data_from_path

activation_dict = {'0': tf.nn.relu, '1': tf.nn.leaky_relu}  # Declare global dictionary

# These are important parameters
run_params = {'batch_size': 4,
              'checkpoint_min': 5,
              'early_stop_step': 10000,
              'input_path': './data/cifar10.tfrecords',
              'result_path': '/home/pasin/Documents/Pasin/model/cifar10_hyper',
              'config_path': '',
              'steps': 100}

model_configs = {'learning_rate': 0.0001,
                 'dropout_rate': 0.1,
                 'activation': tf.nn.relu,
                 'channels': [16, 16, 32, 32, 32, 32, 32, 32, 32, 2048, 2048]
                 }


def run(model_params={}):
    # Add exception in case params is missing
    if len(model_params['channels']) != 11:
        raise Exception("Number of channels not correspond to number of layers [Need size of 11, got %s]"
                        % len(model_params['channels']))

    # Type in file name
    train_data_path = run_params['input_path'].replace('.tfrecords', '') + '_train.tfrecords'
    eval_data_path = run_params['input_path'].replace('.tfrecords', '') + '_eval.tfrecords'
    print("Getting training data from %s" % train_data_path)
    print("Saved model at %s" % run_params['result_path_new'])

    tf.logging.set_verbosity(tf.logging.INFO)  # To see some additional info
    # Setting checkpoint config
    my_checkpoint_config = tf.estimator.RunConfig(
        save_checkpoints_secs=run_params['checkpoint_min'] * 60,
        # save_summary_steps=pareval_data_pathams['checkpoint_min'] * 10,
        keep_checkpoint_max=10,
        log_step_count_steps=500,
        session_config=tf.ConfigProto(allow_soft_placement=True)
    )
    # Or set up the model directory
    #   estimator = DNNClassifier(
    #       config=tf.estimator.RunConfig(
    #           model_dir='/my_model', save_summary_steps=100),
    classifier = tf.estimator.Estimator(
        model_fn=my_model,
        params=model_params,
        model_dir=run_params['result_path_new'],
        config=my_checkpoint_config
    )
    train_hook = tf.contrib.estimator.stop_if_no_decrease_hook(classifier, "loss", run_params['early_stop_step'])
    train_spec = tf.estimator.TrainSpec(
        input_fn=lambda: train_input_fn(train_data_path, batch_size=run_params['batch_size']),
        max_steps=run_params['steps'], hooks=[train_hook])
    eval_spec = tf.estimator.EvalSpec(
        input_fn=lambda: eval_input_fn(eval_data_path, batch_size=16), steps=None,
        start_delay_secs=0, throttle_secs=0)
    # classifier.train(input_fn=lambda: train_input_fn(train_data_path, batch_size=params['batch_size']),
    #     max_steps=params['steps'], hooks=[train_hook])
    # eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(eval_data_path, batch_size=32))

    eval_result = tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)
    print("Eval result:")
    print(eval_result)
    try:
        accuracy = eval_result[0]['accuracy']
        global_step = eval_result[0]['global_step']
    except TypeError:
        print("Warning, does receive evaluation result")
        accuracy = 0
        global_step = 0

    predictions = classifier.predict(input_fn=lambda: eval_input_fn(eval_data_path, batch_size=1))

    images, expected = get_data_from_path(eval_data_path)
    predict_score = ['Prediction']
    probability_score = ['Probability']
    label_score = ['Label']
    for pred_dict, expec in zip(predictions, expected):
        # print(pred_dict)
        print("Score" + str(pred_dict['score']))
        class_id = pred_dict['score'][0]
        probability = pred_dict['probabilities'][class_id]
        print("Actual score: %s, Predicted score: %s with probability %s" % (expec, class_id, probability))
        predict_score.append(class_id)
        label_score.append(expec)
        probability_score.append(probability)

    # predict_result = zip(label_score, predict_score, probability_score)

    predict_result = zip(label_score, predict_score, probability_score)
    # print(eval_result[0]['accuracy'])
    # print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
    return accuracy, global_step, predict_result


dim_learning_rate = Real(low=1e-5, high=1e-2, prior='log-uniform', name='learning_rate')
dim_dropout_rate = Real(low=0, high=0.875, name='dropout_rate')
dim_activation = Categorical(categories=['0', '1'],
                             name='activation')
dim_channel = Integer(low=1, high=4, name='channels')
dimensions = [dim_learning_rate,
              dim_dropout_rate,
              dim_activation,
              dim_channel]
default_parameters = [1e-3, 0.125, '1', 2]


@use_named_args(dimensions=dimensions)
def fitness(learning_rate, dropout_rate, activation, channels):
    """
    Hyper-parameters:
    learning_rate:     Learning-rate for the optimizer.
    dropout_rate:
    activation:        Activation function for all layers.
    channels
    """
    # Create the neural network with these hyper-parameters
    print("Learning_rate, Dropout_rate, Activation, Channels = %s, %s, %s, %s" % (
        learning_rate, dropout_rate, activation, channels))
    channels_full = [i * channels for i in [16, 16, 32, 16, 16, 16, 16, 16, 16, 512, 512]]
    name = run_params['result_path'] + "/" + datetime.datetime.now().strftime("%Y%m%d_%H_%M_%S") + "/"
    # name = ("%s/learning_rate_%s_dropout_%s_activation_%s_channels_%s/"
    #         % (run_params['result_path'], round(learning_rate, 6), dropout_rate, activation, channels))
    run_params['result_path_new'] = name
    md_config = {'learning_rate': learning_rate,
                 'dropout_rate': dropout_rate,
                 'activation': activation_dict[activation],
                 'channels': channels_full}
    accuracy, global_step, result = run(md_config)
    # accuracy = run_hyper_parameter_wrapper(learning_rate, dropout_rate, activation, channels)

    # Save necessary info to csv file, as reference
    info_dict = run_params.copy()
    info_dict['comments'] = "Try hyper search on cifar10"
    info_dict['learning_rate'] = learning_rate
    info_dict['dropout_rate'] = dropout_rate
    info_dict['activation'] = activation
    info_dict['channels'] = channels
    info_dict['steps'] = global_step
    info_dict['accuracy'] = accuracy
    with open(name + "config.csv", "w") as csvfile:
        writer = csv.writer(csvfile)
        for key, val in info_dict.items():
            writer.writerow([key, val])
    with open(name + "result.csv", "w") as csvfile:
        writer = csv.writer(csvfile)
        for row in result:
            writer.writerow(row)
    return -accuracy


search_result = gp_minimize(func=fitness,
                                dimensions=dimensions,
                                acq_func='EI',  # Expected Improvement.
                                n_calls=11,
                                x0=default_parameters)
print(search_result)
print("Best hyper-parameters: %s" % search_result.x)
searched_parameter = sorted(list(zip(search_result.func_vals, search_result.x_iters)))
print("All hyper-parameter searched: %s" % searched_parameter)
hyperparameter_filename = "hyperparameters_" + datetime.datetime.now().strftime("%Y%m%d_%H_%M_%S") + ".csv"
new_data = []
field_name = ['accuracy', 'learning_rate', 'dropout_rate', 'activation', 'channels']
print(searched_parameter)
for i in searched_parameter:
    data = {field_name[0]: i[0] * -1,
                field_name[1]: i[1][0],
                field_name[2]: i[1][1],
                field_name[3]: i[1][1],
                field_name[4]: i[1][1]}
    new_data.append(data)
with open(run_params['result_path'] + '/' + hyperparameter_filename, 'w', newline='') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames=field_name)
    writer.writeheader()
    writer.writerows(new_data)
# space = search_result.space
# print("Best result: %s" % space.point_to_dict(search_result.x))

print("train.py completed")


In [ ]:
print("Func vals: %s \n" % search_result.func_vals)
print("Hyper para: %s \n" % search_result.x_iters)
print("Combined: %s" % list(zip(search_result.func_vals, search_result.x_iters)))

In [ ]:
searched_parameter_2 = (list(zip(search_result.func_vals, search_result.x_iters)))
i = searched_parameter_2[0]
data_2 = {field_name[0]: i[0] * -1,
                field_name[1]: i[1][0],
                field_name[2]: i[1][1],
                field_name[3]: i[1][1],
                field_name[4]: i[1][1]}
print(data_2)

In [1]:
# This file is to convert stl file and csv file into cross_section image
# After running this, use imgtotfrecord.py to turn images into tfrecord

# Import Libraries
import time
from ImportData2D import get_label, get_file_name, save_plot
from stlSlicer import getSlicer, slicecoor, rotatestl
import numpy as np

v = '1.2.0'
# Initial version: Based on main.ipynb
# 1.1: Implemented ignore data that has problem
# 1.2: Now save image with their own name
print("pre_processing.py version: " + str(v))

degree = list([0, 45, 90, 135])

def main():
    # Get data and transformed to cross-section image
    name_dir, image_name = get_file_name(folder_name='../global_data/', file_name="PreparationScan.stl")
    label, label_name = get_label("Taper_Sum", "median", double_data=True, one_hotted=False, normalized=False)
    # Number of data should be the same as number of label
    if image_name != label_name:
        print(image_name)
        print(label_name)
        diff = list(set(image_name).symmetric_difference(set(label_name)))
        raise Exception("ERROR, image and label not similar: %d images, %d labels. Possible missing files: %s"
                        % (len(image_name), (len(label_name)), diff))

    augment_config = list([False, True])  # Original data once, Augmented once
    # Prepare two set of list, one for data, another for augmented data
    stl_points = list()
    stl_points_augmented = list()
    error_file_names = list()  # Names of file that cannot get cross-section image
    min_point = 1000
    max_point = 0
    for i in range(len(name_dir)):
        for augment in augment_config:
            points = getSlicer(name_dir[i], 0, degree, augment, axis=1)
            if points is None:  # If the output has error, remove label of that file
                error_file_names.append(image_name[i])
                index = label_name.index(image_name[i])
                label_name.pop(index)
                label.pop(index * 2)
                label.pop(index * 2)  # Do it again if we double the data
                break
            else:
                if len(points[0]) > max_point:
                    max_point = len(points[0])
                if len(points[0]) < min_point:
                    min_point = len(points[0])
                if augment:
                    stl_points_augmented.append(points)
                else:
                    stl_points.append(points)

    # The output is list(examples) of list(degrees) of numpy array (N*2 coordinates)
    print("Finished with %d examples, %d augmented examples" % (len(stl_points), len(stl_points_augmented)))
    print("Number of score received: %d (Originally: %d" % (len(label), len(label) / 2))

    print("Max point: %s, min  point: %s" % (max_point, min_point))
    # augment_num = int(len(label)/len(label_name))
    # label_name_aug = [val for val in label_name for _ in range(augment_num)]
    return stl_points, stl_points_augmented, label_name, error_file_names, degree


def save_image(stl_points, stl_points_augmented, label_name, error_file_names):
    # Save data as png image
    png_name = "PreparationScan" + "_0"
    save_plot(stl_points, "./data/cross_section", png_name, label_name, degree)
    print("Finished saving first set of data")
    # Save again for augmented data
    png_name = "PreparationScan" + "_1"
    save_plot(stl_points_augmented, "./data/cross_section", png_name, label_name, degree)
    print("Finished saving second set of data")

    # Save names which has defect on it, use when convert to tfrecord
    with open('./data/cross_section/error_file.txt', 'w') as filehandle:
        for listitem in error_file_names:
            filehandle.write('%s\n' % listitem)


def save_movement(stl_points, label_name):
    new_stl_points = []
    for stl_point_sample in stl_points:
        new_points_sample = []
        for d in range(len(degree)):
            points = stl_point_sample[0]
        new_stl_points.append(new_points_sample)



points, points_aug, lbl_name, err_name, deg = main()  # Output points as list[list[numpy]] (example_data, degrees, points)
print(type(points))
print(len(points))
print(type(points[0]))
print(len(points[0]))
print(type(points[0][0]))
print(np.shape(points[0][0]))
# save_image(points, points_aug, lbl_name, err_name)
print("pre_processing.py: done")


ImportData2D.py version: 2.3.1
stlSlicer.py version: 1.8.1
pre_processing.py version: 1.2.0
get_file_name: Import from /home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/global_data, searching for file name with PreparationScan.stl
get_file_name: Uploaded 365 file names
get_label: Import from ../global_data/Ground Truth Score_new.csv
get_label: Upload non one-hotted label completed (as a list): 730 examples
getSlicer: /home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/global_data/Original Format Data/304121-2/PreparationScan.stl has problem getting cross-section. Possible hole appeared in model
getSlicer: /home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/global_data/Original Format Data/84119-2/PreparationScan.stl has problem getting cross-section. Possible hole appeared in model
getSlicer: /home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/global_data/Original Format Data/94106-2/P

In [23]:
stl_points = points
new_stl_points = []
for stl_point_sample in stl_points:  # stl_point_sample is one example of stl_points
    new_points_sample = []
    for stl_point_image in stl_point_sample:  # stl_point_image is one degree of cross-section
        difference = stl_point_image[1:, :] - stl_point_image[0:-1, :]  # Find difference between each position
        new_points_sample.append(difference)
    new_stl_points.append(new_points_sample)

In [24]:
print(type(new_stl_points))
print(len(new_stl_points))
print(type(new_stl_points[0]))
print(len(new_stl_points[0]))
print(type(new_stl_points[0][0]))
print(np.shape(new_stl_points[0][0]))

<class 'list'>
362
<class 'list'>
4
<class 'numpy.ndarray'>
(352, 2)


In [25]:
print(new_stl_points[0][0])

[[-6.7410000e-05  3.0381200e-03]
 [-7.9602000e-04  3.5868770e-02]
 [-1.2950000e-03  1.0284902e-01]
 [-2.9505600e-03  5.3032030e-02]
 [-5.2922700e-03  1.2944522e-01]
 [-1.7759400e-03  7.7626380e-02]
 [-1.5305900e-03  7.1442220e-02]
 [-4.0421400e-03  1.6210612e-01]
 [-4.2452600e-03  6.7096730e-02]
 [-2.7566200e-03  1.1339957e-01]
 [-6.7806600e-03  1.4826855e-01]
 [-5.2958100e-03  9.5722900e-02]
 [-4.2223400e-03  1.2270431e-01]
 [-3.5535000e-04  2.5624560e-02]
 [-1.0479700e-03  2.6030330e-02]
 [-2.1326300e-03  1.1554921e-01]
 [-1.3645100e-03  4.6904350e-02]
 [-5.6744300e-03  1.0090443e-01]
 [-7.6224000e-04  1.4843120e-02]
 [-1.3419000e-04  4.4982600e-03]
 [-8.7140000e-05  3.7094100e-03]
 [-9.2631600e-03  1.4819503e-01]
 [-1.6463100e-03  5.2617150e-02]
 [-4.7271400e-03  1.4477225e-01]
 [-3.6569300e-03  7.7267230e-02]
 [-9.9820000e-04  2.7121810e-02]
 [-2.1981000e-03  3.0852160e-02]
 [-8.4874100e-03  8.9498350e-02]
 [-2.6326800e-03  3.2619140e-02]
 [-9.7921600e-03  9.3374320e-02]
 [-5.68446